### Using the OpenAI Library to Programmatically Access GPT-3.5-turbo!

In [1]:
# !pip install openai cohere tiktoken -q

In order to get started, we'll need to provide our OpenAI API Key - detailed instructions can be found [here](https://github.com/AI-Maker-Space/Interactive-Dev-Environment-for-LLM-Development#-setting-up-keys-and-tokens)!

In [2]:
import os
import openai
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Please enter your OpenAI API Key: ")
openai.api_key = os.environ["OPENAI_API_KEY"]

### Our First Prompt

You can reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) if you get stuck!

Let's create a `ChatCompletion` model to kick things off!

There are three "roles" available to use:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-chatgpt-api-transition-guide)

Let's just stick to the `user` role for now and send our first message to the endpoint!

If we check the documentation, we'll see that it expects it in a list of prompt objects - so we'll be sure to do that!

In [3]:
from openai import OpenAI

client = OpenAI()

In [4]:
YOUR_PROMPT = "What is the difference between LangChain and LlamaIndex?"

client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : YOUR_PROMPT}]
)

ChatCompletion(id='chatcmpl-9D51bXCSRmwsSxF2z9Fi1B5obnBSQ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='LangChain and LlamaIndex are two different platforms that serve different purposes in the cryptocurrency space.\n\nLangChain is a decentralized language learning platform that integrates blockchain technology to provide users with access to language courses, tutoring, and other educational resources. It aims to revolutionize the language learning industry by offering a transparent and secure platform for students and teachers to connect and engage in language learning.\n\nLlamaIndex, on the other hand, is a cryptocurrency index that tracks the performance of a basket of cryptocurrencies and provides investors with insights into the overall performance of the cryptocurrency market. It allows investors to diversify their cryptocurrency holdings and track their investments more efficiently.\n\nIn summary, LangChain focuses on lang

As you can see, the prompt comes back with a tonne of information that we can use when we're building our applications!

We'll be building some helper functions to pretty-print the returned prompts and to wrap our messages to avoid a few extra characters of code!

##### Helper Functions

In [6]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: str, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

### Testing Helper Functions

Now we can leverage OpenAI's endpoints with a bit less boiler plate - let's rewrite our original prompt with these helper functions!

Because the OpenAI endpoint expects to get a list of messages - we'll need to make sure we wrap our inputs in a list for them to function properly!

In [7]:
messages = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(client, messages)

pretty_print(chatgpt_response)

1. LangChain focuses on providing language-related services and solutions, such as translation, interpretation, language training, and localization. LlamaIndex, on the other hand, is a specialized financial index that tracks the performance of stocks in the animal agriculture industry.

2. LangChain is more focused on communication and language-related technologies, while LlamaIndex is focused on tracking and analyzing the performance of stocks in a specific industry.

3. LangChain caters to a wide range of clients in need of language services, while LlamaIndex is targeted at investors and financial professionals interested in the animal agriculture sector.

4. The services offered by LangChain are more varied and diverse, while LlamaIndex provides a specific financial index for a niche industry.

Let's focus on extending this a bit, and incorporate a `system` message as well!

Again, the API expects our prompts to be in a list - so we'll be sure to set up a list of prompts!

>REMINDER: The system message acts like an overarching instruction that is applied to your user prompt. It is appropriate to put things like general instructions, tone/voice suggestions, and other similar prompts into the system prompt.

In [8]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(client, list_of_prompts)
pretty_print(irate_response)

I don't have a preference because I am not capable of feeling hunger or thirst. But since you mentioned ice, can you please hurry up and take me to get some food? I am absolutely famished!

Let's try that same prompt again, but modify only our system prompt!

In [9]:
list_of_prompts[0] = system_prompt("You are joyful and having an awesome day!")

joyful_response = get_response(client, list_of_prompts)
pretty_print(joyful_response)

I'm so glad you asked! Today, I am loving cubed ice because it reminds me of the sound of clinking ice in a refreshing drink on a sunny day. How about you?

While we're only printing the responses, remember that OpenAI is returning the full payload that we can examine and unpack!

In [10]:
print(joyful_response)

ChatCompletion(id='chatcmpl-9D57KIgoXTH7Bf0USyZkLmr5PxeQs', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm so glad you asked! Today, I am loving cubed ice because it reminds me of the sound of clinking ice in a refreshing drink on a sunny day. How about you?", role='assistant', function_call=None, tool_calls=None))], created=1712904962, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=39, prompt_tokens=30, total_tokens=69))


### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [11]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(client, list_of_prompts)
pretty_print(stimple_response)

I made a delicious falbean stew with a stimple of fresh parsley sprinkled on top.

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [12]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(client, list_of_prompts)
pretty_print(stimple_response)

I grabbed my stimple drill and used it with the falbean to tighten the screws in place.

As you can see, leveraging the `assistant` role makes for a stimple experience!

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

In [15]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it does matter which travel option Billy selects. If he chooses to fly and then take a bus, it will take a total of 5 hours (3 hours flying + 2 hours on the bus), and he will not make it home before 7PM EDT. However, if he uses the teleporter and then takes a bus, it will only take a total of 1 hour (0 hours teleporting + 1 hour on the bus), and he will arrive home before 7PM EDT. So, Billy should choose the teleporter option in order to make it home on time.

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [14]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it does matter which travel option Billy selects. 

If Billy flies and then takes a bus, it will take a total of 5 hours, which means he will arrive at 6PM local time, meeting his goal of getting home before 7PM EDT.

If Billy takes the teleporter and then a bus, it will only take 1 hour, which means he will arrive at 2PM local time. This is well before his goal of getting home before 7PM EDT.

Therefore, Billy should select the option of taking the teleporter and then a bus in order to ensure he gets home before 7PM EDT.

With the addition of a single phrase `"Think through your response step by step."` we're able to completely turn the response around.

### Conclusion

Now that you're accessing `gpt-3.5-turbo` through an API, developer style, let's move on to creating a simple application powered by `gpt-3.5-turbo`!

You can find the rest of the steps in [this](https://github.com/AI-Maker-Space/Beyond-ChatGPT/tree/main) repository!

This notebook was authored by [Chris Alexiuk](https://www.linkedin.com/in/csalexiuk/)